# Named Entity Recognition Model
# Examples


In [1]:
import spacy
import pandas as pd
import numpy as np
from spacy import displacy
%load_ext autoreload
%autoreload 2
from data_prep import search_keyword_loc_old,filter_spans,convert_to_spacy

In [2]:


text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.render(doc, style="ent")


In [3]:
df=pd.read_excel('data_masking.xlsx')

<ipython-input-3-02f3202cf741>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df=pd.read_excel('data_masking.xlsx')


In [4]:
df['description'][1]

'[First Amendment effective 01-May-17 to DOW For Compare Test - Org Confirmations effective 12-Sep-16 to RR Agreement effective 01-Jan-12 to SN Agreement effective 01-Jan-12 between ABC and  XYZ, Inc.]'

In [5]:
for i in df['description']:
    text = i
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char,ent.label_)
    displacy.render(doc, style="ent")

12 66 68 CARDINAL
01-Jan-12 102 111 DATE
01-Jan-12 138 147 DATE
ABC 156 159 ORG
XYZ 164 167 ORG
months 241 247 DATE
& Renewal Notes 260 275 ORG


First Amendment 1 16 ORG
01 27 29 CARDINAL
Compare Test - Org Confirmations 48 80 ORG
12 91 93 CARDINAL
01-Jan-12 127 136 DATE
01-Jan-12 163 172 DATE
ABC 181 184 ORG
XYZ, Inc. 190 199 ORG


31 69 71 CARDINAL
01-Jan-26 105 114 TIME
01-Jan-12 141 150 DATE
ABC 159 162 ORG
XYZ 168 171 ORG
External Party Current Name 219 246 ORG
0069082-XYZ 248 259 PERSON
90];[Notice 310 321 CARDINAL
months 381 387 DATE
& Renewal Notes 400 415 ORG


Tenth Amendment 1 16 LAW
01 27 29 CARDINAL
31 85 87 CARDINAL
01-Jan-12 121 130 DATE
01-Jan-12 157 166 DATE
ABC 175 178 ORG
XYZ, Inc. 184 193 ORG


Tenth Amendment 1 16 LAW
07-Jul-16 27 36 DATE
31 85 87 CARDINAL
01-Jan-12 121 130 DATE
01-Jan-12 157 166 DATE
ABC 175 178 ORG
XYZ, Inc. 183 192 ORG


BT Application Management Services 32 66 ORG
01 77 79 CARDINAL
01-Jun-13 120 129 DATE
ABC 138 141 ORG
XYZ Inc.]];[Contract No 146 169 ORG
External Party Current Name 196 223 ORG
90];[Notice 284 295 CARDINAL
months 355 361 DATE
& Renewal Notes 374 389 ORG


100279 10 16 DATE
01 46 48 CARDINAL
BCBT Application Management Services 67 103 ORG
01 114 116 CARDINAL
01-Jun-13 157 166 DATE
ABC 175 178 ORG
XYZ Inc. 183 191 ORG
1 212 213 CARDINAL
1 year 235 241 DATE


01-Dec-20 27 36 DATE
BCBT Application Management Services 48 84 ORG
01 95 97 CARDINAL
01-Jun-13 138 147 DATE
ABC 156 159 ORG
XYZ Inc. 164 172 ORG


01 45 47 CARDINAL
ABC 63 66 ORG
XYZ (Canada 71 82 ORG
TAT-32998];[External Party Current Name 108 147 ORG
30];[Notice 212 223 CARDINAL
months 285 291 DATE
& Renewal Notes 303 318 ORG


Data Integrity Services 23 46 ORG
03-Oct-16 57 66 DATE
01-Jan-12 93 102 DATE
01-Jan-12 129 138 DATE
ABC 147 150 ORG
XYZ 156 159 ORG
months 234 240 DATE
& Renewal Notes 253 268 ORG


# Contract Description: 
The information we would like to extract: 


|e_contract_type|e_service_type|effective_date| e_parent_contract_type|parent_effective_date| internal_party| external_party_name| <div style="width:100px"> Renew <div style="width:100px">   |
|--------------|-----------------|-------------------|-----------------|-----------------|-----------------|-----------------|-----------------------|
|Important Property, we mainly focus on **published contracts** with: **Admendment, SOW, Master Contract, Tower Contract and Confidentiality** type|Based on which **agreement** (EX: Network Service agreement)|Made on "date" excuted on "date", Start date|We might have **multiple** parent contracts but with **parent-child relationship**|made on "date" excuted on "date"|Our CIBC Department/ Team| Supplier Name| The renew period (in month, in year, in week)|

    
**Note**:<br>
    1. For Confidentiality: We don't care about the effective date, and no parent contract <br>
    2. For Admendment Contract: We need effective date, parent contract name and internnal/ external parties.<br>
    3. Bridge letter: is a tempeorary contract, we don't need to extract information from them.


In [2]:

import pandas as pd
from tqdm.notebook import tqdm
import spacy
from spacy.tokens import DocBin

def search_keyword_loc_old(texts,dict_keywords_label):
    '''
    This Function is used to:
    Locate the keyword we want
    dict_indext: The dictionary including the keyword and their corresponding label
    return: the list with input text data with location of keywords(start and end position)
    '''
    list_res=[]
    for text in texts:
        dict_index={"entity":list()}
        for i in range(len(dict_keywords_label)):

            for key, label in dict_keywords_label[i].items():
                if text.find(key)!=-1:
                    dict_index["entity"].append((text.find(key),
                                                    (len(key)+text.find(key)),   label))

        list_res.append((text,dict_index))
    return list_res



def filter_spans(spans):
#  """Filter a sequence of spans and remove duplicates or overlaps. Useful for
#  creating named entities (where one token can only be part of one entity) or
#  when merging spans with `Retokenizer.merge`. When spans overlap, the (first)
#  longest span is preferred over shorter spans.

#  spans (iterable): The spans to filter.
#  RETURNS (list): The filtered spans.
#  """
    get_sort_key = lambda span: (span.end - span.start, -span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
     # Check for end - 1 here because boundaries are inclusive
     if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    result = sorted(result, key=lambda span: span.start)
    return result
    get_sort_key = lambda span: (span.end - span.start, -span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
     # Check for end - 1 here because boundaries are inclusive
     if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    result = sorted(result, key=lambda span: span.start)
    return result


def convert_to_spacy(data,path="/Users/fandi/Desktop/cibc_test/corups/train.spacy"):
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object
    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        
        for start, end, label in annot["entity"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        ents = filter_spans(ents)
        doc.set_ents(ents) # label the text with the ents
        db.add(doc)

    db.to_disk(path) # save the docbin object

In [44]:
dict_index_tr=[{'Statement of Work':'Contract_type','effective 01-May-17':"Start_date","For Compare Test - Org Confirmations":"Service type",
            'RR Agreement':'Parent_contract',"SN Agreement":"Parent_contract",
          "01-Jan-12":"Parent_Start_date"," 12];[Term & Renewal Notes: ]":"Renew",
            "ABC":"Internal Party","XYZ":"External Party"},{'Tenth Amendment':'Contract_type','effective 01-May-18':"Start_date",
            'Statement of Work':'Parent_contract',"for Compare Test effective": "Service type","RR Agreement":"Parent_contract",
          "31-Mar-16":"Parent_Start_date","01-Jan-12":"Parent_Start_date","SN Agreement":"Parent_contract","01-Jan-12":"Parent_Start_date",
            "ABC":"Internal Party","XYZ":"External Party"},{"Master Network Services Agreement":"Contract_type","effective 01-Oct-11":"Start_date",
                                                           "ABC":"Internal Party","XYZ (Canada) Ltd":"External Party",
                                                            "(in months): 6];[Term & Renewal Notes: ]": "Renew"}]
dict_index_t=[{'First Amendment':'Contract_type','effective 01-01-May-17':"Start_date","DOW": "Parent_contract",
            'For Compare Test - Org':'Service type',"RR Agreement":"Parent_contract",
          "12-Sep-16":"Parent_Start_date","RR Agreement":"Parent_contract","01-Jan-12":"Parent_Start_date",
            "SN Agreement":"Parent_contract","01-Jan-12":"Parent_Start_date",
            "ABC":"Internal Party","XYZ":"External Party"},{"Master Network Services Agreement":"Contract_type","effective 01-Oct-11":"Start_date",
                                                           "ABC":"Internal Party","XYZ (Canada) Ltd":"External Party",
                                                            "(in months): 6];[Term & Renewal Notes: ]": "Renew"}]

In [45]:
train=search_keyword_loc_old([df['description'][0],df['description'][3],df['description'][8]],dict_index_tr)


In [46]:
test=search_keyword_loc_old([df['description'][1],df['description'][8]],dict_index_t)

In [49]:
convert_to_spacy(data=train,path="/Users/fandi/Desktop/cibc_test/corups/train.spacy")
convert_to_spacy(data=test,path="/Users/fandi/Desktop/cibc_test/corups/dev.spacy")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [50]:
!python -m spacy init config --lang en --pipeline ner C:/Users/fandi/Desktop/cibc_test/config.cfg --force

[!] To generate a more effective transformer-based config (GPU-only), install
the spacy-transformers package and re-run this command. The config generated now
does not use transformers.
[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
C:\Users\fandi\Desktop\cibc_test\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2021-05-31 11:17:42.027605: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-05-31 11:17:42.027674: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Error processing line 7 of C:\Users\fandi\Anaconda3\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\fandi\Anaconda3\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


In [51]:
!python -m spacy train C:/Users/fandi/Desktop/cibc_test/config.cfg \
    --output C:/Users/fandi/Desktop/cibc_test/corups/  \
        --paths.train C:/Users/fandi/Desktop/cibc_test/corups/train.spacy \
            --paths.dev C:/Users/fandi/Desktop/cibc_test/corups/dev.spacy \
                --training.eval_frequency 15 --training.max_steps 150
#                
            

[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     28.50    0.00    0.00    0.00    0.00
  7      15          4.78    771.27    0.00    0.00    0.00    0.00
 15      30        314.59    617.47   26.09   33.33   21.43    0.26
 22      45          5.73    255.92   52.17   66.67   42.86    0.52
 30      60       1122.75    726.75   37.04   38.46   35.71    0.37
 37      75        285.52    413.11   50.00   60.00   42.86    0.50
 45      90       1693.20    305.44   75.86   73.33   78.57    0.76
 52     105          4.64     91.10   75.86   73.33   78.57    0.76
 60     120          2.53     35.57   74.07   76.92   71.43 

2021-05-31 11:17:56.562173: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-05-31 11:17:56.562294: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Error processing line 7 of C:\Users\fandi\Anaconda3\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\fandi\Anaconda3\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored
[2021-05-31 11:18:03,254] [INFO] Set up nlp object from config
[2021-05-31 11:18:03,272] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-05-31 11:18:03,278] [INFO] Created vocabulary
[2021-05-31 11:18:03,278] [INFO] Finished initializing nlp object
[2021-05-31 11:18:03,532] [INFO] Initialized pipeline componen

In [52]:
nlp1 = spacy.load(R"C:\Users\fandi\Desktop\cibc_test\corups\model-best") #Load the model

In [53]:
def test_model(data,model):
    doc_list=[]
    dict_res={""}
    for i in data:
        doc=model(i)
        doc_list.append(model(i))
        for ent in doc.ents:
            print(ent.text, ent.label_)
        spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter
        print("="*60)
    return doc_list

In [54]:
doc_list=test_model(df['description'],nlp1)

Statement of Work Contract_type
For Compare Test - Org Confirmations Service type
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


First Amendment Contract_type
effective 01-May-17 Start_date
DOW For Parent_contract
Compare Test - Org Confirmations Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party


Statement of Work Contract_type
for Compare Test effective Service type
31-Jun-27 Parent_Start_date
RR Agreement Parent_contract
01-Jan-26 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


Tenth Amendment Contract_type
effective 01-May-18 Start_date
Statement of Work Contract_type
for Compare Test effective Service type
31-Mar-16 Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party


Tenth Amendment Contract_type
effective 07-Jul-16 Start_date
Statement of Work Contract_type
for Compare Test effective Service type
31-Mar-16 Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party


DOW for BT Application Management Services Contract_type
effective 01-Dec-16 Start_date
effective 01-Jun-13 Start_date
ABC Internal Party
XYZ External Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


Parent = 100279] Contract_type
Second Amendment Contract_type
effective 01-Dec-19 Start_date
DOW Internal Party
for BCBT Application Management Services effective Service type
01-Dec-16 Parent_Start_date
effective 01-Jun-13 Start_date
ABC Internal Party
XYZ External Party
1 additional External Party
1 year] Renew


Third Amendment Contract_type
effective 01-Dec-20 Start_date
DOW Internal Party
for BCBT Application Management Services effective Service type
01-Dec-16 Parent_Start_date
effective 01-Jun-13 Start_date
ABC Internal Party
XYZ Inc.] External Party


Master Network Services Agreement Contract_type
effective 01-Oct-11 Start_date
ABC Internal Party
XYZ (Canada) External Party
(in months): 6];[Term & Renewal Notes: ] Renew


Statement of Work Contract_type
for Data Integrity Services effective Service type
03-Oct-16 Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


In [6]:
import numpy as np
dict_final={"Contract_type":[],"Service type":[],"Start_date":[],
            "Parent_contract":[],"Parent_Start_date":[]
            ,"Renew":[],"Internal Party":[],"External Party":[]}
col_name=[]
counter=0
for doc in doc_list:
    dict_doc={"Contract_type":[],"Service type":[],"Start_date":[],
            "Parent_contract":[],"Parent_Start_date":[],
              "Renew":[],"Internal Party":[],"External Party":[]}
    list_label=[]
    for ent in doc.ents:            
        list_label.append(ent.label_)
   
    col_name.append(list_label)
    na_list=set(dict_final.keys())-set(col_name[counter])
    counter=counter+1
        
    for n in na_list:
        dict_doc[n].append(np.nan)
    for ent in doc.ents:
        dict_doc[ent.label_].append(ent.text)
    for key in dict_final.keys():
        dict_final[key].append(dict_doc[key])
    

In [63]:
dict_final['Contract_type'][4][0]

'Tenth Amendment'

In [29]:
df.description[5]

'[Contract Description: [DOW for BT Application Management Services effective 01-Dec-16 to AMS Tower Agreement effective 01-Jun-13 between ABC and XYZ Inc.]];[Contract No.: ];[CMG Contract No.: ];[External Party Current Name: 0062-XYZ INC];[Term Type: Fixed];[Notice Period (in days): 90];[Notice Timeframe: Prior to Expiration Date];[Renewal Interval (in months): 12];[Term & Renewal Notes: ]'

In [64]:
df1=pd.DataFrame(dict_final)

In [69]:
df1['Contract_type'][0][0]

'Statement of Work'

In [70]:
df1.columns=["e_contract_type","e_service_type","e_effective_date" ,"e_parent_contract_type","e_parent_effective_date","Renew","internal_party" ,"external_party_name"]

In [71]:
df1

,e_contract_type,e_service_type,e_effective_date,e_parent_contract_type,e_parent_effective_date,Renew,internal_party,external_party_name
0,[Statement of Work],[For Compare Test - Org Confirmations],[nan],"[RR Agreement, SN Agreement]",[01-Jan-12],"[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],[XYZ]
1,[First Amendment],[nan],[effective 01-May-17],"[DOW For, RR Agreement, SN Agreement]","[Compare Test - Org Confirmations, 01-Jan-12]",[nan],[ABC],[XYZ]
2,[Statement of Work],[for Compare Test effective],[nan],"[RR Agreement, SN Agreement]","[31-Jun-27, 01-Jan-26]","[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],"[XYZ, XYZ]"
3,"[Tenth Amendment, Statement of Work]",[for Compare Test effective],[effective 01-May-18],"[RR Agreement, SN Agreement]","[31-Mar-16, 01-Jan-12]",[nan],[ABC],[XYZ]
4,"[Tenth Amendment, Statement of Work]",[for Compare Test effective],[effective 07-Jul-16],"[RR Agreement, SN Agreement]","[31-Mar-16, 01-Jan-12]",[nan],[ABC],[XYZ]
5,[DOW for BT Application Management Services],[nan],"[effective 01-Dec-16, effective 01-Jun-13]",[nan],[nan],"[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],"[XYZ, XYZ]"
6,"[Parent = 100279], Second Amendment]",[for BCBT Application Management Services effe...,"[effective 01-Dec-19, effective 01-Jun-13]",[nan],[01-Dec-16],[1 year]],"[DOW, ABC]","[XYZ, 1 additional]"
7,[Third Amendment],[for BCBT Application Management Services effe...,"[effective 01-Dec-20, effective 01-Jun-13]",[nan],[01-Dec-16],[nan],"[DOW, ABC]",[XYZ Inc.]]
8,[Master Network Services Agreement],[nan],[effective 01-Oct-11],[nan],[nan],[(in months): 6];[Term & Renewal Notes: ]],[ABC],[XYZ (Canada)]
9,[Statement of Work],[for Data Integrity Services effective],[nan],"[RR Agreement, SN Agreement]","[03-Oct-16, 01-Jan-12]","[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],[XYZ]


In [82]:
df1['sub_contract_type']=[np.nan]*len(df1)
counter=0
for i in df1['e_contract_type']:

    for item in i:
        if "amendment" in item.lower():
            print(item.lower().replace("amendment",""))
            df1["sub_contract_type"][counter]=item.lower().replace("amendment","")
    counter=counter+1
            
            

first 
tenth 
tenth 
second 
third 


C:\Users\fandi\Anaconda3\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [83]:
df1

,e_contract_type,e_service_type,e_effective_date,e_parent_contract_type,e_parent_effective_date,Renew,internal_party,external_party_name,sub_contract_type
0,[Statement of Work],[For Compare Test - Org Confirmations],[nan],"[RR Agreement, SN Agreement]",[01-Jan-12],"[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],[XYZ],NaN
1,[First Amendment],[nan],[effective 01-May-17],"[DOW For, RR Agreement, SN Agreement]","[Compare Test - Org Confirmations, 01-Jan-12]",[nan],[ABC],[XYZ],first
2,[Statement of Work],[for Compare Test effective],[nan],"[RR Agreement, SN Agreement]","[31-Jun-27, 01-Jan-26]","[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],"[XYZ, XYZ]",NaN
3,"[Tenth Amendment, Statement of Work]",[for Compare Test effective],[effective 01-May-18],"[RR Agreement, SN Agreement]","[31-Mar-16, 01-Jan-12]",[nan],[ABC],[XYZ],tenth
4,"[Tenth Amendment, Statement of Work]",[for Compare Test effective],[effective 07-Jul-16],"[RR Agreement, SN Agreement]","[31-Mar-16, 01-Jan-12]",[nan],[ABC],[XYZ],tenth
5,[DOW for BT Application Management Services],[nan],"[effective 01-Dec-16, effective 01-Jun-13]",[nan],[nan],"[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],"[XYZ, XYZ]",NaN
6,"[Parent = 100279], Second Amendment]",[for BCBT Application Management Services effe...,"[effective 01-Dec-19, effective 01-Jun-13]",[nan],[01-Dec-16],[1 year]],"[DOW, ABC]","[XYZ, 1 additional]",second
7,[Third Amendment],[for BCBT Application Management Services effe...,"[effective 01-Dec-20, effective 01-Jun-13]",[nan],[01-Dec-16],[nan],"[DOW, ABC]",[XYZ Inc.]],third
8,[Master Network Services Agreement],[nan],[effective 01-Oct-11],[nan],[nan],[(in months): 6];[Term & Renewal Notes: ]],[ABC],[XYZ (Canada)],NaN
9,[Statement of Work],[for Data Integrity Services effective],[nan],"[RR Agreement, SN Agreement]","[03-Oct-16, 01-Jan-12]","[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],[XYZ],NaN


In [84]:
df1

,e_contract_type,e_service_type,e_effective_date,e_parent_contract_type,e_parent_effective_date,Renew,internal_party,external_party_name,sub_contract_type
0,[Statement of Work],[For Compare Test - Org Confirmations],[nan],"[RR Agreement, SN Agreement]",[01-Jan-12],"[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],[XYZ],NaN
1,[First Amendment],[nan],[effective 01-May-17],"[DOW For, RR Agreement, SN Agreement]","[Compare Test - Org Confirmations, 01-Jan-12]",[nan],[ABC],[XYZ],first
2,[Statement of Work],[for Compare Test effective],[nan],"[RR Agreement, SN Agreement]","[31-Jun-27, 01-Jan-26]","[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],"[XYZ, XYZ]",NaN
3,"[Tenth Amendment, Statement of Work]",[for Compare Test effective],[effective 01-May-18],"[RR Agreement, SN Agreement]","[31-Mar-16, 01-Jan-12]",[nan],[ABC],[XYZ],tenth
4,"[Tenth Amendment, Statement of Work]",[for Compare Test effective],[effective 07-Jul-16],"[RR Agreement, SN Agreement]","[31-Mar-16, 01-Jan-12]",[nan],[ABC],[XYZ],tenth
5,[DOW for BT Application Management Services],[nan],"[effective 01-Dec-16, effective 01-Jun-13]",[nan],[nan],"[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],"[XYZ, XYZ]",NaN
6,"[Parent = 100279], Second Amendment]",[for BCBT Application Management Services effe...,"[effective 01-Dec-19, effective 01-Jun-13]",[nan],[01-Dec-16],[1 year]],"[DOW, ABC]","[XYZ, 1 additional]",second
7,[Third Amendment],[for BCBT Application Management Services effe...,"[effective 01-Dec-20, effective 01-Jun-13]",[nan],[01-Dec-16],[nan],"[DOW, ABC]",[XYZ Inc.]],third
8,[Master Network Services Agreement],[nan],[effective 01-Oct-11],[nan],[nan],[(in months): 6];[Term & Renewal Notes: ]],[ABC],[XYZ (Canada)],NaN
9,[Statement of Work],[for Data Integrity Services effective],[nan],"[RR Agreement, SN Agreement]","[03-Oct-16, 01-Jan-12]","[(in months):, 12];[Term & Renewal Notes: ]]",[ABC],[XYZ],NaN


In [33]:
df1.to_csv("Example_description.csv")

# Contract Overview
Keywords we want to extract:
1. Supplier name: External party full name
2. Internal contract number
3. Parent contract number
4. Effective date:
5. Parent contract effective date
6. Responsibility (objective)
7. Core fees : Contract fee with currency sign

**Note**: We will focus on master contracts, tower, SOW and Admendment Contracts

In [60]:
df_new=pd.read_excel("masking_overview4.xlsx")

<ipython-input-60-3e08830aef6a>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_new=pd.read_excel("masking_overview4.xlsx")


In [12]:
df_new['contract_overview'][5]

'[Contract Overview: An amendment to ICN 66827 to engage TestSupplier for SoW services in support of the Incentive Compensation Management application.];[Sourcing Rationale: Provide a hosted service for compensation calculations and management to Wealth investment advisors.];[Key T&C’s: \uf02d\tSecurity Risk Management provisions are governed by Article 13 of the EMA. \n\uf02d\tABC Audit Rights are governed by Article 11.02 of the EMA; however, ABC data is housed in a known third party data centre and TestSupplier does not currently have audit rights. While TestSupplier does intend to eventually bring the solution in-house, this is an ongoing plan. TestSupplier has agreed to our standard audit rights for all other parts of the services and will be opened to audit. \n\uf02d\tService Level Agreement established for key business requirements are set out in the ASP Tower with specific call-outs in the SOW\n\uf02d\tLimitation of Liability for damages are governed by Article 16 of the EMA an

In [17]:
dict_index_tr_ov_new=[{'Amendment':'contract type',"Master Services Cost bubbletea Agreement":"contract detail","5-Year Term":"length"},
                  {'bubble tea price agreement':'contract type',"purchase additional Customer Based licenses":"contract detail","$278,666.72":"price term",
                   "support cap for first and second annual renewal":"renew term"},{'Golden Gate':"contract type",
                    "Processor Perpetual licenses":"contract detail","$33333.33":"price term"},
                      {"purchase agreement":"contract type","Enterprise Security & Risk Services team":"contract detail","$20,400":"price term"}]
dict_index_t_ov_new=[{'NDA':'contract type',"Bubbletea project":"contract detail"},
                     {"amendment" :"contract type","engage TestSupplier for SoW services":"contract detail","$ 1,666.56":"price term"}]

In [105]:
df['overview'][9]


"[Contract Overview: Beta Test (BPO) of Data Integrity Services.  BPO DOW effective Oct 3, 2016 to RR Agreement effective Jan 1, 2012 to EMA effective Jan 1, 2012 between ABC and XYZ Inc.];[Sourcing Rationale: Execute a BPO DOW under the existing XYZ Tower Agreement];[Key T&C's: This DOW is governed by the XYZ RR Agreement and is bound by all terms under the EMA.  The 5% Annual Productivity Improvement will not apply until either: a) an Amendment to this DOW is executed, or b) a new DOW under the RR Agreement is executed, in either case equivalent to 7 FTE.];[Pricing Terms: $M (incl. tax) of transition fees to be paid over 3 Milestones.  Ongoing services fees as per RR Agreement rate card.]"

In [48]:
dict_index_tr_ov=[{'Beta Test of Compare Test (CT) - Org Confirmations':'contract type',
                   "XYZ RR Agreement":"parent contract name",
            'As per legal summary provided by external counsel':'price term',
                   "The 500% Annual Productivity Improvement": "responsibility"},
                  {'24-month AMS agreement under the AMS Tower':'contract type','effective June 1, 2013':"effective date",
            'XYZ will be supporting the D, T, P and M applications in BBT.':'responsibility',
                   "EMA":"parent contract name", "effective January 1, 2012": "parent effective date",
                   "The DOW Initial Term is 24 months with two additional one-year renewal options.": "duration"
                   "Fixed monthly fee of $M (plus taxes) per month for the first 36 months; $M (plus taxes) per month for the following 36 months": "price term"}]
dict_index_t_ov=[{'Beta Test (BPO) of Data Integrity Services':'contract type','effective Oct 3, 2016':"effective date",
                  "RR Agreement": "parent contract name","effective Jan 1, 2012": "parent effective date",
                  "EMA":"parent contract name","effective Jan 1, 2012":"parent effective date"}]

In [10]:
for i in df_new['contract_overview']:
    print("="*60)
    print(i)

[Contract Overview: This is an Amendment and Restatement of the Master Services Cost bubbletea Agreement and service schedules (the Attachments).  This agreement covers services provided between and amongst ABC's Canadian and US legal entities for which there are (or will be) cost recovery "Transfer Pricing" between the providers and recipients of the services.  This update, part of the Cardinal project, further incorporates ABC restaurant USA as both a service provider and a service recipient.   The Attachments herein have been organized based on ABCs Functional Groups and Strategic Business Units (as applicable) in India and US Region and each of these have been formally approved by their respective Senior Executives.];[Sourcing Rationale: Regulatory];[Key T&C’s: 5-Year Term, auto renews for 1-year
Services Schedule outlines the services provided, along with standards, reporting and Transfer pricing methodology and agreements that are implicit in this.];[Pricing Terms: Transfer Prici

In [22]:
train_ov=search_keyword_loc_old([i for i in df_new["contract_overview"][0:4]],dict_index_tr_ov_new)
test_ov=search_keyword_loc_old([df_new['contract_overview'][4],df_new['contract_overview'][5]],dict_index_t_ov_new)

In [23]:
convert_to_spacy(data=train_ov,path="/Users/fandi/Desktop/cibc_test/corups/train_ov.spacy")
convert_to_spacy(data=test_ov,path="/Users/fandi/Desktop/cibc_test/corups/dev_ov.spacy")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
test_ov

[('[Contract Overview: NDA for Bubbletea project.\nAn integrated technical solution; (i) to replace soon to be unsupported core operating/trading systems and (ii) develop Cayrestaurant’s own USD General Ledger framework.]',
  {'entity': [(20, 23, 'contract type')]}),
 ('[Contract Overview: An amendment to ICN 66827 to engage TestSupplier for SoW services in support of the Incentive Compensation Management application.];[Sourcing Rationale: Provide a hosted service for compensation calculations and management to Wealth investment advisors.];[Key T&C’s: \uf02d\tSecurity Risk Management provisions are governed by Article 13 of the EMA. \n\uf02d\tABC Audit Rights are governed by Article 11.02 of the EMA; however, ABC data is housed in a known third party data centre and TestSupplier does not currently have audit rights. While TestSupplier does intend to eventually bring the solution in-house, this is an ongoing plan. TestSupplier has agreed to our standard audit rights for all other parts 

In [24]:
!python -m spacy init config --lang en --pipeline ner C:/Users/fandi/Desktop/cibc_test/config.cfg --force

[!] To generate a more effective transformer-based config (GPU-only), install
the spacy-transformers package and re-run this command. The config generated now
does not use transformers.
[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
C:\Users\fandi\Desktop\cibc_test\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2021-06-01 09:45:34.176370: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-06-01 09:45:34.176600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [25]:
!python -m spacy train C:/Users/fandi/Desktop/cibc_test/config.cfg \
    --output C:/Users/fandi/Desktop/cibc_test/corups/  \
        --paths.train C:/Users/fandi/Desktop/cibc_test/corups/train_ov.spacy \
            --paths.dev C:/Users/fandi/Desktop/cibc_test/corups/dev_ov.spacy \
                --training.eval_frequency 15 --training.max_steps 150

[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     83.00    0.00    0.00    0.00    0.00
  3      15          6.24   1309.83    0.00    0.00    0.00    0.00
  7      30        163.23    170.05    0.00    0.00    0.00    0.00
 11      45        322.18    202.50    0.00    0.00    0.00    0.00
 15      60       6819.73    916.31    0.00    0.00    0.00    0.00
 18      75        739.65    680.72   28.57   50.00   20.00    0.29
 22      90         97.70    144.05   28.57   50.00   20.00    0.29
 26     105        951.05    253.58   28.57   50.00   20.00    0.29
 30     120       2243.41    272.37   28.57   50.00   20.00 

2021-06-01 09:46:14.221963: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-06-01 09:46:14.222144: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2021-06-01 09:46:38,630] [INFO] Set up nlp object from config
[2021-06-01 09:46:38,694] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-06-01 09:46:38,717] [INFO] Created vocabulary
[2021-06-01 09:46:38,717] [INFO] Finished initializing nlp object
[2021-06-01 09:46:39,711] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [26]:
nlp2 = spacy.load(R"C:\Users\fandi\Desktop\cibc_test\corups\model-best") #Load the model

In [27]:
def test_model(data,model):
    doc_list=[]
    dict_res={""}
    for i in data:
        doc=model(i)
        doc_list.append(model(i))
        for ent in doc.ents:
            print(ent.text, ent.label_)
        spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter
        print("="*60)
    return doc_list

0     [Contract Overview: This is an Amendment and R...
1     [Contract Overview: This bubble tea price agre...
2     [Contract Overview: Purchase of eighty (80) Te...
3     [Contract Overview: Intake # 1000753\nThis is ...
4     [Contract Overview: NDA for Bubbletea project....
5     [Contract Overview: An amendment to ICN 66827 ...
6     [Contract Overview: Randomname is an Order Man...
7     [Contract Overview: ABC Standard 2-way Confide...
8     [Contract Overview: Moonbeam has been providin...
9     [Contract Overview: Intake # 1888888\nThis is ...
10    [Contract Overview: This is an amendment to re...
11    [Contract Overview: Intake # 222222\nExtension...
12    [Contract Overview: Amendment to the SW Licens...
13    [Contract Overview: Intake # 1000569\nWwins ha...
14    [Contract Overview: TestSupplier4 will perform...
15    [Contract Overview: Addendum/SOW to add new co...
16    [Contract Overview: Two way NDA between ABC an...
17    [Contract Overview: Service Agreement for 

In [32]:
doc_list=test_model(df_new['contract_overview'],nlp2)

Amendment contract type
Master Services Cost bubbletea Agreement contract detail
5-Year Term, auto renews for 1-year
Services Schedule outlines the services provided, along with standards, reporting and Transfer pricing methodology and agreements that are implicit in this.];[Pricing Terms: Transfer Pricing based on cost allocations through transfer pricing agreements, guided by the Allocation Notes described in the Attachments.  Generally Cost Reimbursement and consistent with Regulation W principles.] length


bubble tea contract type
purchase additional Customer Based licenses contract detail
$278,666.72 price term
support cap contract type


Golden Gate contract type
Processor Perpetual licenses contract detail
$33333.33 price term


purchase agreement contract type
Enterprise Security & Risk Services team contract detail
$20,400 price term


C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\lib\site-packages\spacy\displacy\__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


...

$33,000 price term


$55,555.00 price term


mid March 2019];[Key contract detail


$777,000 price term
$900.00 price term


$666,000.00 price term



|Stanford NER Model|	Data|	Data Source|	Model Description|	Entities|Accuracy Score|
|-----------------|-------------|------------|-----------------------|-------------|------------------|
|7 class|	MUC 6 and MUC 7(~318 news articles in MUC 6)|	newswire|	CRFClassifier|	Location, Person, Organization, Money, Percent, Date, Time|0.89 ± 0.11|

### Check detailed documentation: https://nlp.stanford.edu/software/CRF-NER.html#Download

In [65]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
jar="C:/Users/fandi/Desktop/cibc_test/stanford-ner.jar"
st = StanfordNERTagger('C:/Users/fandi/Desktop/cibc_test/stanford-ner-4.2.0/stanford-ner-2020-11-17/classifiers/english.conll.4class.distsim.crf.ser.gz',jar) 
text="Fandi want to drink bubbletea"


In [68]:
list_res=[]
for i in df_new['contract_overview']:
    tokenized_text = word_tokenize(i)
    classified_text = st.tag(tokenized_text)
    list_res.append(classified_text)
    print("finished one extraction")
    

finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction
finished one extraction


# Coreference Resolution
### 1 Neuralcoref
### 2 allennlp_models

In [78]:
import neuralcoref
text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
print(doc._.coref_clusters)

In [74]:
from stanfordnlp.server import CoreNLPClient

In [79]:
import stanfordnlp
stanfordnlp.download('en')   # This downloads the English models for the neural pipeline


In [84]:
nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English
doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc.sentences[0].print_dependencies()

In [83]:
import stanza
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse') # This sets up a default neural pipeline in English


2021-06-01 12:34:24 INFO: Downloading default packages for language: en (English)...
2021-06-01 12:34:27 INFO: File exists: C:\Users\fandi\stanza_resources\en\default.zip.
2021-06-01 12:34:37 INFO: Finished downloading models and saved to C:\Users\fandi\stanza_resources.
2021-06-01 12:34:37 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-06-01 12:34:37 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2021-06-01 12:34:37 INFO: Use device: cpu
2021-06-01 12:34:37 INFO: Loading: tokenize
2021-06-01 12:34:37 INFO: Loading: pos
2021-06-01 12:34:38 INFO: Loading: lemma
2021-06-01 12:34:38 INFO: Loading: depparse
2021-06-01 12:34:39 INFO: Done loading processors!


In [122]:
for i in df_new['contract_overview']:
    word = nlp(i)
    print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')


In [123]:
print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')


id: 1	word: [	head id: 3	head: Overview	deprel: punct
id: 2	word: Contract	head id: 3	head: Overview	deprel: compound
id: 3	word: Overview	head id: 0	head: root	deprel: root
id: 4	word: :	head id: 3	head: Overview	deprel: punct
id: 5	word: This	head id: 8	head: Amendment	deprel: nsubj
id: 6	word: is	head id: 8	head: Amendment	deprel: cop
id: 7	word: an	head id: 8	head: Amendment	deprel: det
id: 8	word: Amendment	head id: 3	head: Overview	deprel: appos
id: 9	word: and	head id: 10	head: Restatement	deprel: cc
id: 10	word: Restatement	head id: 8	head: Amendment	deprel: conj
id: 11	word: of	head id: 17	head: Agreement	deprel: case
id: 12	word: the	head id: 17	head: Agreement	deprel: det
id: 13	word: Master	head id: 14	head: Services	deprel: compound
id: 14	word: Services	head id: 17	head: Agreement	deprel: compound
id: 15	word: Cost	head id: 17	head: Agreement	deprel: compound
id: 16	word: bubbletea	head id: 17	head: Agreement	deprel: compound
id: 17	word: Agreement	head id: 8	head: Amendm

In [96]:
from allennlp_models import pretrained

In [98]:
model=pretrained.load_predictor("coref-spanbert")

error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\lm-masked-language-model.json as plain json


downloading:   0%|          | 0/1345986155 [00:00<?, ?B/s]

error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\AppData\Local\Temp\tmp7vasu21a\config.json as plain json


Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [102]:
df_new['contract_overview'][0]

'[Contract Overview: This is an Amendment and Restatement of the Master Services Cost bubbletea Agreement and service schedules (the Attachments).  This agreement covers services provided between and amongst ABC\'s Canadian and US legal entities for which there are (or will be) cost recovery "Transfer Pricing" between the providers and recipients of the services.  This update, part of the Cardinal project, further incorporates ABC restaurant USA as both a service provider and a service recipient.   The Attachments herein have been organized based on ABCs Functional Groups and Strategic Business Units (as applicable) in India and US Region and each of these have been formally approved by their respective Senior Executives.];[Sourcing Rationale: Regulatory];[Key T&C’s: 5-Year Term, auto renews for 1-year\nServices Schedule outlines the services provided, along with standards, reporting and Transfer pricing methodology and agreements that are implicit in this.];[Pricing Terms: Transfer Pr

In [104]:
data=model.predict(df_new['contract_overview'][0])

In [112]:
from IPython.core.display import display, HTML
# Add css styles and js events to DOM, so that they are available to rendered html
import pandas as pd
display(HTML(open('visualization/highlight.css').read()))
display(HTML(open('visualization/highlight.js').read()))

import visualization

In [115]:
html = visualization.render(data, allen=True, jupyter=False)
# data.append({'sample_idx': idx, 
#              'model': 'AllenNlp',
#              'annotation': html})

In [117]:
HTML(html)

In [120]:
data2=model.predict(df_new['contract_overview'][2])
html = visualization.render(data2, allen=True, jupyter=False)
HTML(html)

In [124]:
data3=model.predict(df_new['contract_overview'][4])
html = visualization.render(data3, allen=True, jupyter=False)
HTML(html)

In [127]:
data3=model.predict(df_new['contract_overview'][6])
html = visualization.render(data3, allen=True, jupyter=False)
HTML(html)

In [130]:
df=pd.read_excel('data_masking.xlsx')

<ipython-input-130-02f3202cf741>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df=pd.read_excel('data_masking.xlsx')


In [132]:
data3=model.predict(df['overview'][0])
html = visualization.render(data3, allen=True, jupyter=False)
HTML(html)